In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
%cd /content/drive/MyDrive/NLP_Project/es_de/es_to_de/nmt/
!ls

/content/drive/.shortcut-targets-by-id/1EIMToi0LeMeMkQhiUquuEGwIMzoZSKbN/NLP_Project/es_de/es_to_de/nmt
compute-bleu.py				     Europarl.de.translated
config.yaml				     Europarl.de.translated2
Europarl.de-es.de			     Europarl.de.translated2.desubword
Europarl.de-es.de-filtered.de		     Europarl.de.translated.desubword
Europarl.de-es.de-filtered.de.subword	     LICENSE
Europarl.de-es.de-filtered.de.subword.dev    models
Europarl.de-es.de-filtered.de.subword.test   MT-Preparation
Europarl.de-es.de-filtered.de.subword.train  README
Europarl.de-es.es			     run
Europarl.de-es.es-filtered.es		     source.model
Europarl.de-es.es-filtered.es.subword	     source.vocab
Europarl.de-es.es-filtered.es.subword.dev    target.model
Europarl.de-es.es-filtered.es.subword.test   target.vocab
Europarl.de-es.es-filtered.es.subword.train  train.log
Europarl.de-es.xml


In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: Europarl.de-es.es-filtered.es.subword.train
        path_tgt: Europarl.de-es.de-filtered.de.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: Europarl.de-es.es-filtered.es.subword.dev
        path_tgt: Europarl.de-es.de-filtered.de.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 3

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 25000

# Default: 10000 - Run validation after n steps
valid_steps: 500

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 2000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 3
dec_layers: 3
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-19 22:23:09,683 INFO] Counter vocab from -1 samples.
[2023-06-19 22:23:09,683 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-19 22:25:07,694 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=609)

[2023-06-19 22:25:07,855 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=648)

[2023-06-19 22:25:07,997 INFO] Counters src: 50369
[2023-06-19 22:25:07,997 INFO] Counters tgt: 50334


In [ ]:
# Check if the GPU is active
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [ ]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14998.8125 out of: 15101.8125


In [ ]:
# Train the NMT model
!onmt_train -config config.yaml

[2023-06-19 22:25:12,855 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-19 22:25:12,856 INFO] Parsed 2 corpora from -data.
[2023-06-19 22:25:12,856 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-19 22:25:13,444 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '▁de', ',', '▁la', '.', '▁que', '▁en']
[2023-06-19 22:25:13,445 INFO] The decoder start token is: <s>
[2023-06-19 22:25:13,446 INFO] Building model...
[2023-06-19 22:25:15,237 INFO] Switching model to float32 for amp/apex_amp
[2023-06-19 22:25:15,238 INFO] Non quantized layer compute is fp16
[2023-06-19 22:25:16,338 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50000, 512, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer

In [ ]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.fren_step_5000.pt -src Europarl.de-es.es-filtered.es.subword.test -output Europarl.de.translated -gpu 0 -min_length 1

[2023-06-20 16:56:43,090 INFO] Loading checkpoint from models/model.fren_step_5000.pt
[2023-06-20 16:58:42,412 INFO] PRED SCORE: -0.5134, PRED PPL: 1.67 NB SENTENCES: 4000


In [ ]:
!onmt_translate -model models/model.fren_step_15000.pt -src Europarl.de-es.es-filtered.es.subword.test -output Europarl.de.translated2 -gpu 0 -min_length 1

[2023-06-20 16:58:45,080 INFO] Loading checkpoint from models/model.fren_step_15000.pt
[2023-06-20 17:00:38,842 INFO] PRED SCORE: -0.4409, PRED PPL: 1.55 NB SENTENCES: 4000


In [ ]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.de.translated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.2 MB/s eta 0:00:00
Done desubwording! Output: Europarl.de.translated.desubword


In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.de.translated2

Done desubwording! Output: Europarl.de.translated2.desubword


In [ ]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.de-es.de-filtered.de.subword.test

Done desubwording! Output: Europarl.de-es.de-filtered.de.subword.test.desubword


In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-06-20 17:00:50--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py.1’

compute-bleu.py.1   100%[===================>]     957  --.-KB/s    in 0s      

2023-06-20 17:00:50 (36.9 MB/s) - ‘compute-bleu.py.1’ saved [957/957]



In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py Europarl.de-es.de-filtered.de.subword.test.desubword Europarl.de.translated.desubword

Reference 1st sentence: Das ist im Moment das Wichtigste.
MTed 1st sentence: Das ist jetzt wichtig.
BLEU:  24.360715466154428


In [ ]:
!python3 compute-bleu.py Europarl.de-es.de-filtered.de.subword.test.desubword Europarl.de.translated2.desubword

Reference 1st sentence: Das ist im Moment das Wichtigste.
MTed 1st sentence: Darum geht es jetzt.
BLEU:  26.966396527887603
